In [ ]:
import socket

# Create UDP server socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

# Dictionary of valid users (username -> password)
users = {"user1": "pass1", "user2": "pass2", "admin": "admin123"}

# Dictionary to store authenticated clients
authenticated_clients = {}

print("UDP Chat Server with Authentication is running...")

while True:
    data, client_address = server_socket.recvfrom(2048)
    message = data.decode()

    # If client is not authenticated, expect username and password
    if client_address not in authenticated_clients:
        try:
            username, password = message.split(",")  # Expect "username,password"
        except ValueError:
            server_socket.sendto(b"Invalid login format. Use: username,password", client_address)
            continue

        if username in users and users[username] == password:
            authenticated_clients[client_address] = username
            server_socket.sendto(b"Login successful. You can now chat.", client_address)
            print(f"{username} authenticated from {client_address}")
        else:
            server_socket.sendto(b"Invalid credentials. Try again.", client_address)
        continue

    # Process chat messages after authentication
    print(f"{authenticated_clients[client_address]} ({client_address}): {message}")

    # Broadcast message to other authenticated clients
    for client in authenticated_clients:
        if client != client_address:
            server_socket.sendto(f"{authenticated_clients[client_address]}: {message}".encode(), client)
